In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [35]:
categorias=pd.read_csv("../data_raw/categorias.csv")
clientes=pd.read_csv("../data_raw/clientes.csv")
productos=pd.read_csv("../data_raw/productos.csv")
ventas=pd.read_csv("../data_raw/ventas.csv")
metodos_pago=pd.read_csv("../data_raw/metodos_pago.csv")
display(categorias.head())
display(clientes.head())
display(productos.head())
display(ventas.head())
display(metodos_pago.head())

,ID_Categoria,Categoría,Descripción
0,1,Lácteos,"Productos lácteos frescos y procesados, como l..."
1,2,Carnicería,"Carnes frescas y procesadas, como carne de vac..."
2,3,Panadería,"Productos horneados frescos, como pan, factura..."
3,4,Frutas y Verduras,"Frutas y verduras frescas, locales e importada..."
4,5,Congelados,"Productos congelados, como papas fritas, empan..."


,ID_Cliente,Nombre,Apellido,Email,Fecha_Resgistro,Región
0,1,Karisa,Cromett,kcromett0@imageshack.us,19/11/2023,Patagonia
1,2,Lenette,Seabert,lseabert1@yahoo.co.jp,07/05/2023,Patagonia
2,3,Buddy,Silverson,bsilverson2@howstuffworks.com,27/03/2023,Patagonia
3,4,Dan,Parkin,dparkin3@virginia.edu,26/10/2023,Buenos Aires
4,5,Conney,Cassella,ccassella4@who.int,31/03/2023,Centro


,ID_Producto,Nombre_producto,Categoría,Precio_Unitario,Stock
0,1,Leche,Lácteos,"12,24",3327
1,2,Yogur,Lácteos,"5,21",3358
2,3,Queso cremoso,Lácteos,"17,23",3167
3,4,Queso rallado,Lácteos,"19,23",2099
4,5,Manteca,Lácteos,"5,65",4929


,ID_Venta,Fecha,ID_Cliente,ID_Producto,Cantidad,Método_Pago,Estado
0,919,31/01/2024,10,25,5,1,Completa
1,947,31/01/2024,106,5,1,4,Completa
2,1317,31/1/2024,235,25,3,3,Completa
3,1607,31/1/2024,114,15,5,1,Completa
4,2038,31/1/2024,132,2,5,4,Completa


,ID_Metodo,Método,Descripción
0,1,Efectivo,"Pago en dinero en efectivo, sin intermediarios..."
1,2,Tarjeta de Crédito,Pago con tarjetas emitidas por bancos y financ...
2,3,Tarjeta de Débito,Pago con tarjetas que debitán directamente de ...
3,4,Mercado Pago,Plataforma de pagos online que permite realiza...
4,5,Transferencia,Pago realizado a través de una transferencia d...


In [36]:
productos["Precio_Unitario"]=productos["Precio_Unitario"].str.replace(",",".").astype(float)

In [37]:
df=ventas.merge(clientes, on="ID_Cliente",how="inner")
df.drop(columns=["Email"],inplace=True)
df = df.merge(productos[["ID_Producto", "Precio_Unitario"]], on="ID_Producto", how="inner")
df["Cantidad_dinero"]=df["Cantidad"]*df["Precio_Unitario"]
df

,ID_Venta,Fecha,ID_Cliente,ID_Producto,Cantidad,Método_Pago,Estado,Nombre,Apellido,Fecha_Resgistro,Región,Precio_Unitario,Cantidad_dinero
0,919,31/01/2024,10,25,5,1,Completa,Stephenie,Sexty,23/11/2023,Buenos Aires,15.45,77.25
1,947,31/01/2024,106,5,1,4,Completa,Benedikta,Condon,23/06/2023,NEA,5.65,5.65
2,1317,31/1/2024,235,25,3,3,Completa,Cloe,Brun,11/12/2023,NEA,15.45,46.35
3,1607,31/1/2024,114,15,5,1,Completa,Fabien,Roskam,11/04/2023,Centro,3.51,17.55
4,2038,31/1/2024,132,2,5,4,Completa,Cassie,Corish,14/12/2023,Buenos Aires,5.21,26.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3024,954,30/12/2024,44,12,6,4,Completa,Cyrill,Runge,17/09/2023,Buenos Aires,8.12,48.72
3025,1390,30/12/2024,26,31,3,4,Completa,Byrom,Soro,05/07/2023,Centro,5.24,15.72
3026,1519,30/12/2024,246,11,3,3,Completa,Almira,Zottoli,28/09/2023,NEA,11.23,33.69
3027,2147,30/12/2024,231,22,2,1,Pendiente,Rani,Chattelaine,13/12/2023,Patagonia,4.21,8.42


In [38]:
df=df.merge(productos[["ID_Producto","Stock"]], on="ID_Producto", how="inner")

In [39]:
dict_stock=productos.set_index("ID_Producto")["Stock"].to_dict()
print(dict_stock)
cantidad_de_producto_actual=[]
for cantidad, id_producto in zip(df["Cantidad"], df["ID_Producto"]):
    dict_stock[id_producto]-=cantidad
    cantidad_de_producto_actual.append(dict_stock[id_producto])
    if dict_stock[id_producto]<=0:
        print(f"Stock acabado para el producto {id_producto}")
print(dict_stock)
df["Producto_actual_stock"]=cantidad_de_producto_actual
df

{1: 3327, 2: 3358, 3: 3167, 4: 2099, 5: 4929, 6: 5137, 7: 4068, 8: 3140, 9: 4051, 10: 2429, 11: 1726, 12: 1619, 13: 2228, 14: 3125, 15: 4043, 16: 3189, 17: 3383, 18: 4720, 19: 4041, 20: 1541, 21: 2800, 22: 3545, 23: 3731, 24: 2908, 25: 1640, 26: 1688, 27: 4978, 28: 3637, 29: 2455, 30: 1363, 31: 2771, 32: 5057, 33: 2837, 34: 4769, 35: 1437, 36: 4578, 37: 1506, 38: 2182}
{1: 3051, 2: 3085, 3: 2894, 4: 1840, 5: 4629, 6: 4838, 7: 3775, 8: 2820, 9: 3799, 10: 2174, 11: 1472, 12: 1319, 13: 1969, 14: 2908, 15: 3779, 16: 2959, 17: 3084, 18: 4506, 19: 3793, 20: 1314, 21: 2534, 22: 3320, 23: 3426, 24: 2631, 25: 1308, 26: 1306, 27: 4620, 28: 3409, 29: 2102, 30: 1120, 31: 2471, 32: 4725, 33: 2605, 34: 4487, 35: 1131, 36: 4362, 37: 1249, 38: 1861}


,ID_Venta,Fecha,ID_Cliente,ID_Producto,Cantidad,Método_Pago,Estado,Nombre,Apellido,Fecha_Resgistro,Región,Precio_Unitario,Cantidad_dinero,Stock,Producto_actual_stock
0,919,31/01/2024,10,25,5,1,Completa,Stephenie,Sexty,23/11/2023,Buenos Aires,15.45,77.25,1640,1635
1,947,31/01/2024,106,5,1,4,Completa,Benedikta,Condon,23/06/2023,NEA,5.65,5.65,4929,4928
2,1317,31/1/2024,235,25,3,3,Completa,Cloe,Brun,11/12/2023,NEA,15.45,46.35,1640,1632
3,1607,31/1/2024,114,15,5,1,Completa,Fabien,Roskam,11/04/2023,Centro,3.51,17.55,4043,4038
4,2038,31/1/2024,132,2,5,4,Completa,Cassie,Corish,14/12/2023,Buenos Aires,5.21,26.05,3358,3353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3024,954,30/12/2024,44,12,6,4,Completa,Cyrill,Runge,17/09/2023,Buenos Aires,8.12,48.72,1619,1319
3025,1390,30/12/2024,26,31,3,4,Completa,Byrom,Soro,05/07/2023,Centro,5.24,15.72,2771,2471
3026,1519,30/12/2024,246,11,3,3,Completa,Almira,Zottoli,28/09/2023,NEA,11.23,33.69,1726,1472
3027,2147,30/12/2024,231,22,2,1,Pendiente,Rani,Chattelaine,13/12/2023,Patagonia,4.21,8.42,3545,3320


In [40]:
productos_vendidos=df.groupby("ID_Producto")["Producto_actual_stock"].min().reset_index()
productos_vendidos=productos_vendidos.merge(productos[["ID_Producto","Stock"]], on="ID_Producto", how="inner")
productos_vendidos["Cantidad_vendida"]=productos_vendidos["Stock"]-productos_vendidos["Producto_actual_stock"]
productos_vendidos.sort_values("Cantidad_vendida", ascending=False).reset_index(drop=True)

,ID_Producto,Producto_actual_stock,Stock,Cantidad_vendida
0,26,1306,1688,382
1,27,4620,4978,358
2,29,2102,2455,353
3,32,4725,5057,332
4,25,1308,1640,332
5,38,1861,2182,321
6,8,2820,3140,320
7,35,1131,1437,306
8,23,3426,3731,305
9,31,2471,2771,300


In [41]:
productos_vendidos_sorted = productos_vendidos.sort_values("Cantidad_vendida", ascending=False)
fig = go.Figure()

fig.add_trace(go.Bar(
    x=productos_vendidos_sorted["ID_Producto"].astype(str),
    y=productos_vendidos_sorted["Stock"],
    name="Stock Total",
    opacity=1
))


fig.add_trace(go.Bar(
    x=productos_vendidos_sorted["ID_Producto"].astype(str),
    y=productos_vendidos_sorted["Producto_actual_stock"],
    name="Stock Restante",
    opacity=1
))

fig.update_layout(
    title="Stock Total vs Stock Restante por Producto",
    xaxis_title="ID Producto",
    yaxis_title="Cantidad",
    barmode='overlay',
    width=1000,
    height=600

)

fig.show()

In [42]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=productos_vendidos_sorted["ID_Producto"].astype(str),
    y=productos_vendidos_sorted["Cantidad_vendida"],
    name="Cantidad Vendida",
    marker=dict(
        color=productos_vendidos_sorted["Cantidad_vendida"],
        colorscale='Inferno',
        showscale=True,
        colorbar=dict(title="Cantidad")
    ),
    opacity=1
))

fig.update_layout(
    title="Cantidad Vendida por Producto",
    xaxis_title="ID Producto",
    yaxis_title="Cantidad Vendida",
    width=1000,
    height=600
)

fig.show()